# Validation of the model with experimental data

We import the required libraries

In [1]:
import os
import pathlib

import galpynostatic as gp
import numpy as np
import pandas as pd
import scipy.interpolate

DATA_PATH = pathlib.Path(os.path.abspath(os.path.join("..", "datasets")))
PREDICTIONS_PATH = pathlib.Path(os.path.abspath(os.path.join("..", "predictions")))

## Particle size validation

We load the experimental data for validation

In [2]:
validation_df = pd.read_csv(DATA_PATH / "validation_particle_size.csv")
validation_df

,Material,d,dcoeff,k0,c1,SOCexp1,c2,SOCexp2,dpred
0,Graphite,0.000750,1.233000e-10,2.310000e-07,3.0,0.540030,5,0.296843,4.0270
1,LTO,0.000175,6.579000e-12,8.100000e-08,2.0,0.654458,5,0.346546,0.9620
2,LFP,0.000035,2.800000e-13,1.000000e-09,2.0,0.329323,5,0.024868,0.0840
3,LCO,0.002000,5.300000e-09,1.000000e-05,2.0,0.930123,5,0.834509,28.8000
4,LMO,0.000003,3.510000e-14,1.900000e-08,1.0,0.961700,5,0.938762,0.0734
5,LNMO,0.000800,1.200000e-09,1.000000e-06,2.5,0.996566,5,0.976255,13.0000


and make predictions with them

In [3]:
derr = []
spherical = pd.read_csv(DATA_PATH / "simulated_map.csv")

for i, sys in validation_df.iterrows():
    flin = scipy.interpolate.interp1d(
        [sys["c1"], sys["c2"]], [sys["SOCexp1"], sys["SOCexp2"]]
    )
    socexp = flin(4)
    ellexp = gp.utils.logell(4, sys["d"], 3, sys["dcoeff"])
    
    greg = gp.model.GalvanostaticRegressor(d=sys["d"])
    greg.dcoeff_, greg.k0_ = sys["dcoeff"], sys["k0"]
    greg.dcoeff_err_, greg.k0_err_ = None, None
    greg._map = gp.base.MapSpline(spherical)
 
    d0 = sys["d"] if i != 4 else 2 * sys["d"]
    dopt = gp.make_prediction.optimal_particle_size(
        greg, d0=d0, loaded=socexp, cm_to=1
    )
    ellpred = gp.utils.logell(4, dopt, 3, sys["dcoeff"])
    
    dell = np.abs(ellpred - ellexp)
    derr.append(
        10_000 * dell * np.sqrt(np.abs((3 * sys["dcoeff"] * 3600) / (16 * ellpred)))
    )

and we save the predictions

In [4]:
predictions_df = validation_df.copy()
predictions_df["derr"] = np.asarray(derr, dtype=float)
predictions_df[["Material", "dcoeff", "k0", "dpred", "derr"]]

,Material,dcoeff,k0,dpred,derr
0,Graphite,1.233000e-10,2.310000e-07,4.0270,0.166575
1,LTO,6.579000e-12,8.100000e-08,0.9620,0.006824
2,LFP,2.800000e-13,1.000000e-09,0.0840,0.045392
3,LCO,5.300000e-09,1.000000e-05,28.8000,4.448553
4,LMO,3.510000e-14,1.900000e-08,0.0734,0.021130
5,LNMO,1.200000e-09,1.000000e-06,13.0000,5.423920


In [5]:
predictions_df[["Material", "dcoeff", "k0", "dpred", "derr"]].to_csv(
    PREDICTIONS_PATH / "validation_particle_size.csv", index=False
)

These predictions give an upper bound in the estimation of the optimal particle size.

## Charging rate validation

We load the experimental data for validation

In [6]:
validation_df = pd.read_csv(DATA_PATH / "validation_charging_rate.csv")
validation_df

,Material,d,dcoeff,k0,c1,SOCexp1,c2,SOCexp2
0,Graphite,0.000750,1.233000e-10,2.310000e-07,1.0,0.853887,3.0,0.540030
1,LTO,0.000175,6.579000e-12,8.100000e-08,1.0,0.845837,2.0,0.654458
2,LFP,0.000035,2.800000e-13,1.000000e-09,0.5,0.836089,1.0,0.759624
3,LCO,0.002000,5.300000e-09,1.000000e-05,5.0,0.834509,10.0,0.734328
4,LMO,0.000003,3.510000e-14,1.900000e-08,20.0,0.863516,50.0,0.690223
5,LNMO,0.000800,1.200000e-09,1.000000e-06,7.5,0.830797,12.5,0.725181


and make predictions with them

In [7]:
interpolations, predictions = [], []
spherical = pd.read_csv(DATA_PATH / "simulated_map.csv")

for _, sys in validation_df.iterrows():
    # experimental C-rate to reach 0.8 of the SOC
    flin = scipy.interpolate.interp1d(
        [sys["SOCexp1"], sys["SOCexp2"]], [sys["c1"], sys["c2"]]
    )
    c_rate_exp = flin(0.8)
    interpolations.append(c_rate_exp)
    
    # predicted C-rate to reach 0.8 of the SOC
    greg = gp.model.GalvanostaticRegressor(d=sys["d"])
    greg.dcoeff_, greg.k0_ = sys["dcoeff"], sys["k0"]
    greg.dcoeff_err_, greg.k0_err_ = None, None
    greg._map = gp.base.MapSpline(spherical)
    c_rate_pred = gp.make_prediction.optimal_charging_rate(greg, c0=c_rate_exp)
    
    predictions.append(c_rate_pred)

and we save the predictions

In [8]:
predictions_df = validation_df.copy()
predictions_df["c_rate_exp"] = np.asarray(interpolations, dtype=float)
predictions_df["c_rate_pred"] = predictions
predictions_df[["Material", "d", "dcoeff", "k0", "c_rate_exp", "c_rate_pred"]]

,Material,d,dcoeff,k0,c_rate_exp,c_rate_pred
0,Graphite,0.000750,1.233000e-10,2.310000e-07,1.343386,1.359314
1,LTO,0.000175,6.579000e-12,8.100000e-08,1.239510,1.351943
2,LFP,0.000035,2.800000e-13,1.000000e-09,0.735981,0.993012
3,LCO,0.002000,5.300000e-09,1.000000e-05,6.722320,8.439195
4,LMO,0.000003,3.510000e-14,1.900000e-08,30.995653,34.770294
5,LNMO,0.000800,1.200000e-09,1.000000e-06,8.957961,11.111159


In [9]:
predictions_df[
    ["Material", "d", "dcoeff", "k0", "c_rate_exp", "c_rate_pred"]
].to_csv(PREDICTIONS_PATH / "validation_charging_rate.csv", index=False)

These predictions give the difference in the predicted optimal C-rate against the experimental one.